In [10]:
from unsloth import FastLanguageModel, is_bfloat16_supported
from unsloth.chat_templates import train_on_responses_only, get_chat_template, standardize_sharegpt
import torch
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq

In [4]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-bnb-4bit", # Alternatively, could use unsloth/Meta-Llama-3.1-70B-bnb-4bit if we used a large enough machine
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True, # Loading prequantized,
)

model = FastLanguageModel.get_peft_model(
    model,
    # r is an important hyperparameter, governing rank used for LORA - higher value, more parameters and possibly better fit.
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = True, #
    random_state = 3407,
    use_rslora = False,
    loftq_config = None
)

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass


==((====))==  Unsloth 2024.11.10: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: NVIDIA RTX A4000. Max memory: 15.724 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

Unsloth 2024.11.10 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [13]:
def train_llama(data_name, subject_name):

  dataset = load_dataset("json", data_files = f"llm_finetune_{data_name}_{subject_name}.json", split = "train")

  # Standarize sharegpt expects a hardcoded column for "conversations", while openai expects "messages"
  # So rename for LLaMA:
  dataset = dataset.rename_column("messages", "conversations")

  dataset = standardize_sharegpt(dataset)
  dataset = dataset.map(formatting_prompts_func, batched = True)

  trainer = SFTTrainer(
      model = model,
      tokenizer = tokenizer,
      train_dataset = dataset,
      dataset_text_field = "text",
      max_seq_length = 2048,
      data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
      dataset_num_proc = 2,
      packing = False, # Can make training 5x faster for short sequences.
      args = TrainingArguments(
          per_device_train_batch_size = 2,
          gradient_accumulation_steps = 4,
          warmup_steps = 5,
          num_train_epochs = 1, # Set this for 1 full training run.
          learning_rate = 2e-4,
          fp16 = not is_bfloat16_supported(),
          bf16 = is_bfloat16_supported(),
          logging_steps = 1,
          optim = "adamw_8bit",
          weight_decay = 0.01,
          lr_scheduler_type = "linear",
          seed = 3407,
          output_dir = "outputs",
          report_to = "none", # Use this for WandB etc
      ),
  )

  trainer = train_on_responses_only(
      trainer,
      instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
      response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
  )

  trainer_stats = trainer.train()

  model.save_pretrained(f"llama_3_2_3b_tuned_{data_name}_{subject_name}") # Local saving
  tokenizer.save_pretrained(f"llama_3_2_3b_tuned_{data_name}_{subject_name}")

  return(trainer_stats)

In [14]:
check = train_llama("party_id", "trump")

Map:   0%|          | 0/12218 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/12218 [00:00<?, ? examples/s]

Map:   0%|          | 0/12218 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 12,218 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 1,527
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss
1,2.928300
2,2.925800
3,2.835100
4,2.698400
5,2.171000
6,1.749100
7,1.528900
8,1.393500
9,1.507300
10,1.343300


In [15]:
check = train_llama("party_id", "biden")

Generating train split: 0 examples [00:00, ? examples/s]

Standardizing format:   0%|          | 0/4507 [00:00<?, ? examples/s]

Map:   0%|          | 0/4507 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/4507 [00:00<?, ? examples/s]

Map:   0%|          | 0/4507 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 4,507 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 563
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss
1,1.428200
2,1.363900
3,1.233300
4,1.322200
5,1.282100
6,1.236700
7,1.182900
8,1.159200
9,1.148300
10,1.174200
